In [1]:
import cv2
from ultralytics import YOLO
import supervision as sv
import numpy as np
from BlackjackPlayer import BlackjackPlayer


cap = cv2.VideoCapture(1)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

cap_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
cap_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

print(cap_width, cap_height)

640 480


In [ ]:
# Define the zone polygon
ZONE_POLYGON = np.array([
    [0,0],
    [cap_width ,0],
    [cap_width,cap_height // 2],
    [0,cap_height // 2]
])

model = YOLO('./bestCardDetector.pt')

bounding_box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

zone = sv.PolygonZone(polygon=ZONE_POLYGON)
zone_annotator = sv.PolygonZoneAnnotator(zone=zone, color=sv.Color(255,0,0))

while True:

    ret, frame = cap.read()
    if not ret:
        print("Image capture failed")
        break
    

    # result = model(frame, agnostic_nms=True)[0]
    result = model(frame, agnostic_nms=False)[0]
    detections = sv.Detections.from_ultralytics(result)

    # List to store detected cards in the zone
    d_cards_detected = []
    p_cards_detected = []

    d_idx = []
    p_idx = []

    # Process detections
    for i in range(len(detections)):
        xyxy_tensor = detections[i].xyxy
        xyxy = xyxy_tensor.squeeze()
        xmin, ymin, xmax, ymax = xyxy.astype(int)

        classidx = int(detections[i].class_id)
        classname = model.model.names[classidx]
        conf = detections[i].confidence.item()
            
        if zone.trigger(detections[i])[0]:
            d_cards_detected.append(classname)
            d_idx.append(classidx)
        else :
            p_cards_detected.append(classname)
            p_idx.append(classidx)

    computer = BlackjackPlayer(d_idx, p_idx, model.model.names)
    

    
    
    if len(detections) > 0:
        labels = [
            f"{model.model.names[class_id]} {confidence:.2f}"
            for class_id, confidence in zip(detections.class_id, detections.confidence)
        ]
    else:
        labels = []

    annotated_frame = bounding_box_annotator.annotate(
        scene=frame, detections=detections
    )
    annotated_frame = label_annotator.annotate(
        scene=annotated_frame, detections=detections, labels=labels
    )


    # annotated_frame = zone_annotator.annotate(scene=annotated_frame, label=None)
    cv2.rectangle(annotated_frame, (0, 0), (cap_width, cap_height // 2), (0, 0, 255), thickness=2)

     # Display detected cards in the zones
    cv2.rectangle(annotated_frame, (10, 10), (cap_width  // 3, cap_height // 16), (50, 50, 50), cv2.FILLED)
    cv2.putText(annotated_frame, f'Dealer Cards: {", ".join(d_cards_detected)}', (20, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)

    cv2.rectangle(annotated_frame, (10, cap_height // 2 + 10), (cap_width // 3, cap_height // 2 + cap_height // 16), (50, 50, 50), cv2.FILLED)
    cv2.putText(annotated_frame, f'Player Cards: {", ".join(p_cards_detected)}', (20, cap_height // 2 + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)

    cv2.rectangle(annotated_frame, (cap_width // 2, cap_height // 2 + 10), (cap_width // 2 + cap_width // 2 - 10, cap_height // 2 + cap_height // 16), (50, 50, 50), cv2.FILLED)
    cv2.putText(annotated_frame, f'Play: {computer.play()}', (cap_width // 2, cap_height // 2 + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)


    cv2.imshow("detection", annotated_frame)


    key_press = cv2.waitKey(5)
    if key_press == ord('q'):
        break
    elif key_press == ord('p'):
        cv2.imwrite('table.png', annotated_frame)

cv2.destroyAllWindows()


0: 480x640 1 AS, 79.0ms
Speed: 4.0ms preprocess, 79.0ms inference, 179.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 AS, 65.0ms
Speed: 3.0ms preprocess, 65.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)



0: 480x640 1 AS, 62.0ms
Speed: 2.0ms preprocess, 62.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 AS, 45.0ms
Speed: 1.0ms preprocess, 45.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 AS, 43.0ms
Speed: 3.0ms preprocess, 43.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 AS, 37.0ms
Speed: 2.0ms preprocess, 37.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 AS, 36.0ms
Speed: 1.0ms preprocess, 36.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 AS, 36.0ms
Speed: 2.0ms preprocess, 36.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 AS, 35.0ms
Speed: 2.0ms preprocess, 35.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 AS, 35.0ms
Speed: 1.0ms preprocess, 35.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 AS, 35.0ms


In [ ]:
cap.release()